# Fundamental Properties of EMU Radio Sources - Cross-Match Catalogs using NWAY

## Import Libraries

In [4]:
from astropy.io import fits
import numpy as np
from astropy.table import Table

## Import Catalog Data

In [ ]:
emu_catalog_path = '/data/mfonseca/survey_data/EMU_data/EMU_0102-32/EMU_0102-32_1comp.fits'
des_catalog_path = '/data/mfonseca/survey_data/DES_data/DES_in_EMU_0102-32_square.fits'
viking_catalog_path = '/data/mfonseca/survey_data/VIKING_data/VIKING_in_EMU_0102-32.fits'
catwise_catalog_path = '/data/mfonseca/survey_data/CATWISE_data/CATWISE2020_in_EMU_0102-32.fits'

## Prepare the catalogs for NWAY input
The NWAY library has multiple requirements for a catalog to serve as input.
1. The fits data table needs to have a extension name. It is used as a prefix for the columns copied to the output catalog, so each column from each catalog is called "PREFIXOFCATALOG_NAMEOFCOLUMN"
2. The fits header of the catalogs needs to have the SKYAREA keyword. This keyword has the area on the sky in square degrees covered by the catalog. Since I don't have the documentation for EMU data, I must calculate the area using the mosaic image.
3. The position error needs to be a column with a single value, called positional error, so we need to obtain it from the right ascension and declination errors.

### Change extension name
This process is also possible using just the NWAY library. Check page 8 of the NWAY documentation

In [3]:
def change_hdu_name(fits_file, hdu_index, new_name):
    with fits.open(fits_file, mode='update') as hdul:
        # Change the name of the specified HDU
        hdul[hdu_index].header['EXTNAME'] = new_name
        # Save changes to the FITS file
        hdul.flush()

Change extension name of EMU survey to "EMU" and of GAMA-23 survey to "G23"

In [4]:
change_hdu_name(emu_catalog_path, hdu_index=1, new_name="EMU")
change_hdu_name(des_catalog_path, hdu_index=1, new_name="DES")
change_hdu_name(viking_catalog_path, hdu_index=1, new_name="VKG")
change_hdu_name(catwise_catalog_path, hdu_index=1, new_name="CAT")

### Create and assign value to SKYAREA keyword

Create the SKYAREA in the FITS header and assign a value to it.

In [5]:
def create_skyarea_keyword(fits_file, skyarea_value):
    """
    Create the SKYAREA keyword in the FITS header and assign a value to it.

    Parameters:
    fits_file (str): Path to the FITS file.
    skyarea_value (float): Value to assign to the SKYAREA keyword (in square degrees).
    """
    with fits.open(fits_file, mode='update') as hdul:
        header = hdul[1].header
        
        header['SKYAREA'] = skyarea_value
        
        hdul.flush()
        print(f"SKYAREA keyword set to {skyarea_value} in {fits_file}.")

In [6]:
def calculate_skyarea(ra_interval, dec_interval):
    """
    Calculate the area of the sky.

    Parameters:
    ra_interval (list): List of interval of RA in degrees
    dec_interval (list): List of interval of DEC in degrees

    """
    delta_ra_rad  = np.radians(abs(ra_interval[1]-ra_interval[0]))
    delta_dec_rad = np.sin(np.radians(dec_interval[1])) - np.sin(np.radians(dec_interval[0]))

    solid_angle_sr = delta_ra_rad*delta_dec_rad
    solid_angle_deg2 = solid_angle_sr * (180 / np.pi) ** 2

    return solid_angle_deg2

In [ ]:
# Acoording to documentation it is indeed around 30 degrees
area_emu_survey = calculate_skyarea([12.0,19.0], [-35.16,-30.00])
print(area_emu_survey)

30.425887052030664


In [ ]:
area_emu_survey = calculate_skyarea([12.0,19.0], [-35.16,-30.00])

# The limits are derived from the limit imposed when querying the catalogs
area_des_survey = calculate_skyarea([11.00,20.0], [-36.00,-29.00])
area_viking_survey = calculate_skyarea([11.00,20.0], [-36.00,-29.00])
area_catwise_survey = calculate_skyarea([11.00,20.0], [-36.00,-29.00])


create_skyarea_keyword(emu_catalog_path, area_emu_survey)
create_skyarea_keyword(des_catalog_path, area_des_survey)
create_skyarea_keyword(viking_catalog_path, area_viking_survey)
create_skyarea_keyword(catwise_catalog_path, area_catwise_survey)

SKYAREA keyword set to 30.425887052030664 in /home/polaris/Lab_Astro/data/EMU_data/EMU_0102-32/EMU_0102-32_1comp.fits.
SKYAREA keyword set to 53.100621980421366 in /home/polaris/Lab_Astro/data/DES_data/DES_in_EMU_0102-32_square.fits.
SKYAREA keyword set to 53.100621980421366 in /home/polaris/Lab_Astro/data/VIKING_data/VIKING_in_EMU_0102-32.fits.
SKYAREA keyword set to 53.100621980421366 in /home/polaris/Lab_Astro/data/CATWISE_data/CATWISE2020_in_EMU_0102-32.fits.


## Fix Class of Columns in VIKING Survey

In [10]:
# Read the table from the FITS file
table = Table.read(viking_catalog_path, format='fits')

# Ensure the key columns are floats
for col in ['ra', 'dec', 'zPetroMag', 'zPetroMagErr', 'jPetroMag', 'jPetroMagErr', 'yPetroMag', 'yPetroMagErr', 'ksPetroMag', 'ksPetroMagErr', 'hPetroMag', 'hPetroMagErr']:
    if col in table.colnames:
        # Convert column to float, handling strings or mixed types
        table[col] = np.asarray(table[col], dtype=np.float64)
    else:
        raise KeyError(f"Column '{col}' not found in the FITS table.")

# Save the cleaned table
table.write(viking_catalog_path, format='fits', overwrite=True)

print(f"Cleaned FITS table saved to: {viking_catalog_path}")

Cleaned FITS table saved to: /home/polaris/Lab_Astro/data/VIKING_data/VIKING_in_EMU_0102-32.fits


## Fix ID Column in EMU survey

In [ ]:

def create_id_column_from_source_name(fits_file):
    with fits.open(fits_file, mode='update') as hdul:
        data = hdul[1].data

        if 'Source_Name' not in data.columns.names:
            raise ValueError("The FITS file does not contain a 'Source_Name' column.")
        
        if 'ID' in data.columns.names:
            raise ValueError("The FITS file already contains an 'ID' column.")

        source_names = data['Source_Name']

        id_data = [name.split('EMUJ')[-1] if 'EMUJ' in name else '' for name in source_names]

        id_col = fits.Column(name='ID', format='20A', array=id_data)  # '20A' for a string column of max length 20

        new_columns = hdul[1].columns + fits.ColDefs([id_col])
        new_hdu = fits.BinTableHDU.from_columns(new_columns)

        hdul[1] = new_hdu

        hdul.flush()
        print("Created new 'ID' column with the number part of 'Source_Name'.")